In [1]:
import wandb
run = wandb.init()
artifact = run.use_artifact('ain-space-org/wandb-registry-model/gaia calibrated:v0', type='model')
artifact_dir = artifact.download()
artifact_dir

wandb: Currently logged in as: bernddoser to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: WARNING Registries can be linked/fetched using a shorthand form without specifying the organization name. Try using shorthand path format: <my_registry_name>/<artifact_name> or just <my_registry_name> if fetching just the project.
wandb:   1 of 1 files downloaded.  


'/home/doserbd/git/Spherinator/gaia/artifacts/model-qf4roy5i:v0'

In [ ]:
# import yaml
# import importlib

# config = yaml.load(open('model.yaml'), Loader=yaml.FullLoader)
# model_class_path = config["model"]["class_path"]
# module_name, class_name = model_class_path.rsplit(".", 1)
# module = importlib.import_module(module_name)
# model_class = getattr(module, class_name)
# model_init_args = config["model"]["init_args"]
# model = model_class(**model_init_args)

NameError: name 'torch' is not defined

In [4]:
import torch
from spherinator.models import (
    ConsecutiveConv1DLayer,
    ConvolutionalEncoder1DGen,
    DenseModel,
    VariationalAutoencoderPure,
)

cnn_layers=[ConsecutiveConv1DLayer(kernel_size=5,
                                   base_channel_number=16,
                                   pooling=torch.nn.MaxPool1d(2, ceil_mode=True)),
            ConsecutiveConv1DLayer(kernel_size=7,
                                   base_channel_number=32,
                                   pooling=torch.nn.MaxPool1d(2, ceil_mode=True)),
            ConsecutiveConv1DLayer(kernel_size=9,
                                   base_channel_number=64,
                                   pooling=torch.nn.MaxPool1d(2, ceil_mode=True))]
encoder = ConvolutionalEncoder1DGen(input_dim=[1, 343],
                                    output_dim=16,
                                    cnn_layers=cnn_layers)
decoder = DenseModel(layer_dims=[3, 16, 64, 256, 343],
                     output_shape=[1, 343])
model = VariationalAutoencoderPure(encoder=encoder,
                                   decoder=decoder,
                                   z_dim=3,
                                   beta=1.0e-3)

In [ ]:
import torch

ckpt = artifact_dir + "/model.ckpt"
checkpoint = torch.load(ckpt, weights_only=True, map_location="cpu")
model.load_state_dict(checkpoint["state_dict"], strict=False)
model.eval()

export_options = torch.onnx.ExportOptions(dynamic_shapes=True)
onnx = torch.onnx.dynamo_export(model.variational_encoder, torch.randn(1, 1, 343, device="cpu"), export_options=export_options)
onnx.save("encoder.onnx")

onnx = torch.onnx.dynamo_export(model.decoder, torch.randn(1, 3, device="cpu"), export_options=export_options)
onnx.save("decoder.onnx")

/home/doserbd/.cache/pypoetry/virtualenvs/spherinator-zzYPp9oG-py3.12/lib/python3.12/site-packages/torch/onnx/_internal/_exporter_legacy.py:101: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


OnnxExporterError: Failed to export the model to ONNX. Generating SARIF report at 'report_dynamo_export.sarif'. SARIF is a standard format for the output of static analysis tools. SARIF logs can be loaded in VS Code SARIF viewer extension, or SARIF web viewer (https://microsoft.github.io/sarif-web-component/). Please report a bug on PyTorch Github: https://github.com/pytorch/pytorch/issues